# 01 - EDA and Cleaning

本 Notebook 聚焦於：
- 讀取 `data/processed/capture20110817_cleaned.parquet` 檔案
- 分析欄位型別、資料量與缺失值，作為清洗決策依據
- 紀錄後續特徵工程與模型階段的初步觀察


In [5]:
from pathlib import Path
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_DIR = PROJECT_ROOT / "data" / "processed"
DATA_PATH = DATA_DIR / "capture20110817_cleaned.parquet"

if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"找不到 {DATA_PATH}. 請先執行 scripts/load_raw_data.py 產生 Parquet 檔。"
    )


In [6]:
df = pd.read_parquet(DATA_PATH)
row_count, col_count = df.shape
print(f"資料讀取完成：{row_count:,} 列 × {col_count:,} 欄位")
df.head()


資料讀取完成：8,087,512 列 × 11 欄位


,Date_Flow_Start,Duration,Prot,Src_IP_Port,Dst_IP_Port,Flags,Tos,Packets,Bytes,Flows,Label
0,2011-08-17 12:01:01.780,3.12,UDP,188.75.133.98:16200,147.32.86.125:35248,INT,0,304,219158,1,Background
1,2011-08-17 12:01:01.782,4.92,TCP,119.252.172.92:59067,147.32.84.14:80,A_,0,39,2346,1,Background
2,2011-08-17 12:01:01.783,1.00,TCP,90.183.101.168:80,147.32.86.89:1176,FPA_,0,4,2905,1,Background
3,2011-08-17 12:01:01.783,5.00,TCP,217.169.177.82:56950,147.32.85.26:54145,PA_,0,433,27648,1,Background
4,2011-08-17 12:01:01.784,4.99,TCP,147.32.85.26:54145,217.169.177.82:56950,PA_,0,837,1173304,1,Background


## 1. 基本資訊與資料型別


In [9]:
# 記憶體使用量
memory_mb = df.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"記憶體使用: {memory_mb:.2f} MB")
print(f"\n資料型別:")
print(df.dtypes)


記憶體使用: 2655.83 MB

資料型別:
Date_Flow_Start    datetime64[ns]
Duration                  float64
Prot                       object
Src_IP_Port                object
Dst_IP_Port                object
Flags                      object
Tos                         int64
Packets                     int64
Bytes                       int64
Flows                       int64
Label                      object
dtype: object


## 2. 缺失值檢查


In [11]:
missing = (
    df.isna()
    .sum()
    .to_frame(name="missing_count")
    .assign(missing_pct=lambda d: d["missing_count"] / row_count * 100)
    .sort_values("missing_count", ascending=False)
)

if missing["missing_count"].sum() == 0:
    print("✅ 沒有任何缺失值。")
else:
    print("⚠️ 發現缺失值：")
    print(missing[missing["missing_count"] > 0])


✅ 沒有任何缺失值。


## 3. 數值欄位統計


In [18]:
# 識別數值欄位（int64, float64 等）
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# 檢查預期的數值欄位是否都被正確識別
expected_numeric_cols = ["Duration", "Tos", "Packets", "Bytes", "Flows"]
missing_numeric = [col for col in expected_numeric_cols if col in df.columns and col not in numeric_cols]

if missing_numeric:
    print(f"⚠️ 警告：以下欄位應該是數值型但未被識別: {missing_numeric}")
    for col in missing_numeric:
        print(f"  {col}: {df[col].dtype}")
    print()

if len(numeric_cols) > 0:
    print(f"數值欄位 ({len(numeric_cols)} 個): {', '.join(numeric_cols)}")
    print("\n統計摘要：")
    numeric_summary = df[numeric_cols].describe().T
    # 選擇要顯示的統計量
    display_cols = ["count", "mean", "std", "min", "25%", "50%", "75%", "max"]
    available_cols = [col for col in display_cols if col in numeric_summary.columns]
    print(numeric_summary[available_cols])
else:
    print("⚠️ 資料集中沒有數值欄位")


數值欄位 (5 個): Duration, Tos, Packets, Bytes, Flows

統計摘要：
                count      mean        std   min   25%    50%    75%  \
Duration 8,087,512.00      0.65       1.39  0.00  0.00   0.00   0.25   
Tos      8,087,512.00      0.04       2.66  0.00  0.00   0.00   0.00   
Packets  8,087,512.00     14.27     297.88  1.00  1.00   1.00   2.00   
Bytes    8,087,512.00 12,211.87 408,833.36 60.00 74.00 132.00 410.00   
Flows    8,087,512.00      1.00       0.00  1.00  1.00   1.00   1.00   

                    max  
Duration           5.15  
Tos              192.00  
Packets       69,538.00  
Bytes    105,280,532.00  
Flows              1.00  


## 4. 類別欄位分析


In [ ]:
categorical_cols = df.select_dtypes(include=["object", "string"]).columns.tolist()
if len(categorical_cols) > 0:
    print(f"類別欄位 ({len(categorical_cols)} 個): {', '.join(categorical_cols)}")
    print("\n類別欄位分析：")
    
    categorical_summary = []
    for col in categorical_cols:
        unique_count = df[col].nunique()
        value_counts = df[col].value_counts(dropna=False)
        # 最常見的值
        top_value = value_counts.index[0] if len(value_counts) > 0 else None
        # 最常見的值出現次數
        top_count = value_counts.iloc[0] if len(value_counts) > 0 else 0
        # 最常見的值佔總筆數的百分比
        top_pct = (top_count / row_count * 100) if row_count > 0 else 0
        
        categorical_summary.append({
            "column": col,
            "unique_values": unique_count,
            "top_value": top_value,
            "top_count": top_count,
            "top_pct": round(top_pct, 2)
        })
    
    cat_df = pd.DataFrame(categorical_summary)
    cat_df_sorted = cat_df.sort_values("top_pct", ascending=False)
    print(cat_df_sorted.to_string(index=False))
else:
    print("資料集中沒有字串/類別欄位")


類別欄位 (5 個): Prot, Src_IP_Port, Dst_IP_Port, Flags, Label

類別欄位分析：
     column  unique_values      top_value  top_count  top_pct
      Label              3     Background    7317404    90.48
      Flags             62            INT    5330042    65.90
       Prot             16            UDP    5313150    65.70
Dst_IP_Port         823984 147.32.80.9:53     905114    11.19
Src_IP_Port         905590 147.32.80.9:53     904808    11.19


## 5. 標籤分布分析


In [ ]:
if "Label" in df.columns:
    label_counts = df["Label"].value_counts(dropna=False).to_frame(name="count")
    label_counts["percentage"] = (label_counts["count"] / row_count * 100).round(2)
    label_counts
    
    # 檢查標籤不平衡程度
    print(f"\n標籤不平衡分析：")
    if len(label_counts) > 1:
        max_pct = label_counts["percentage"].max()
        min_pct = label_counts["percentage"].min()
        # 最大標籤的數量約為最小標籤的倍數
        imbalance_ratio = max_pct / min_pct if min_pct > 0 else float('inf')
        print(f"最大標籤比例: {max_pct:.2f}%")
        print(f"最小標籤比例: {min_pct:.2f}%")
        print(f"不平衡比例: {imbalance_ratio:.2f}:1")
else:
    print("此資料集沒有 Label 欄位可供分析")



標籤不平衡分析：
最大標籤比例: 90.48%
最小標籤比例: 4.74%
不平衡比例: 19.09:1


## 5.1 重要類別欄位詳細分布分析

In [26]:
# 分析 Label, Flags, Prot 的完整類別分布
important_categorical_cols = ["Label", "Flags", "Prot"]

for col in important_categorical_cols:
    if col in df.columns:
        print("=" * 60)
        print(f"{col} 欄位分布分析")
        print("=" * 60)
        
        value_counts = df[col].value_counts(dropna=False)
        value_counts_df = value_counts.to_frame(name="count")
        value_counts_df["percentage"] = (value_counts_df["count"] / row_count * 100).round(2)
        value_counts_df["cumulative_pct"] = value_counts_df["percentage"].cumsum().round(2)
        
        print(f"總共有 {len(value_counts_df)} 種不同的值")
        print(f"\n完整分布（按數量排序）：")
        print(value_counts_df.to_string())
        
        # 顯示前 5 個最常見的值
        print(f"\n前 5 個最常見的值：")
        top_5 = value_counts_df.head(5)
        for idx, (value, row) in enumerate(top_5.iterrows(), 1):
            print(f"  {idx}. {value}: {row['count']:,} 筆 ({row['percentage']:.2f}%)")
        
        print()

Label 欄位分布分析
總共有 3 種不同的值

完整分布（按數量排序）：
              count  percentage  cumulative_pct
Label                                          
Background  7317404       90.48           90.48
LEGITIMATE   386889        4.78           95.26
Botnet       383219        4.74          100.00

前 5 個最常見的值：
  1. Background: 7,317,404.0 筆 (90.48%)
  2. LEGITIMATE: 386,889.0 筆 (4.78%)
  3. Botnet: 383,219.0 筆 (4.74%)

Flags 欄位分布分析
總共有 62 種不同的值

完整分布（按數量排序）：
           count  percentage  cumulative_pct
Flags                                       
INT      5330042       65.90           65.90
PA_      1165166       14.41           80.31
A_        385828        4.77           85.08
SPA_      261818        3.24           88.32
FA_       207490        2.57           90.89
_FSPA     151327        1.87           92.76
S_        129973        1.61           94.37
FSPA_     122602        1.52           95.89
SA_        75952        0.94           96.83
FPA_       57640        0.71           97.54
RA_        55708 

## 6. 時間範圍分析


In [22]:
if "Date_Flow_Start" in df.columns:
    # 轉換為 datetime（如果還是字串）
    df_time = df["Date_Flow_Start"].copy()
    if df_time.dtype == "object":
        df_time = pd.to_datetime(df_time, errors="coerce")
    
    time_summary = pd.DataFrame({
        "metric": ["最早時間", "最晚時間", "時間跨度"],
        "value": [
            df_time.min(),
            df_time.max(),
            df_time.max() - df_time.min()
        ]
    })
    print(time_summary.to_string(index=False))
    
    # 檢查是否有異常的時間值（例如 1970-01-01，可能是轉換錯誤）
    if df_time.min() < pd.Timestamp("2010-01-01"):
        print(f"\n⚠️ 警告：發現異常早的時間值 {df_time.min()}，可能是資料轉換錯誤")
else:
    print("未找到 Date_Flow_Start 欄位")


metric                      value
  最早時間 2011-08-17 12:01:01.780000
  最晚時間 2011-08-17 17:12:13.904000
  時間跨度     0 days 05:11:12.124000


## 7. 極端值與異常值檢查


In [23]:
# 檢查負值（不應該出現的負數）
negative_checks = {}
for col in numeric_cols:
    if col in df.columns:
        neg_count = (df[col] < 0).sum()
        if neg_count > 0:
            negative_checks[col] = neg_count

if negative_checks:
    print("⚠️ 發現負值：")
    for col, count in negative_checks.items():
        print(f"  {col}: {count:,} 筆 ({count/row_count*100:.2f}%)")
else:
    print("✅ 所有數值欄位都沒有負值")

# 檢查零值（某些欄位不應該為 0）
zero_checks = {}
for col in ["Bytes", "Packets"]:
    if col in df.columns:
        zero_count = (df[col] == 0).sum()
        if zero_count > 0:
            zero_checks[col] = zero_count

if zero_checks:
    print(f"\n⚠️ 發現零值：")
    for col, count in zero_checks.items():
        print(f"  {col}: {count:,} 筆 ({count/row_count*100:.2f}%)")
else:
    print(f"\n✅ Bytes 和 Packets 都沒有零值")


✅ 所有數值欄位都沒有負值

✅ Bytes 和 Packets 都沒有零值


## 8. IP 與 Port 格式檢查


In [24]:
# 檢查 IP:Port 格式是否正確
for col in ["Src_IP_Port", "Dst_IP_Port"]:
    if col in df.columns:
        # 檢查是否包含冒號（IP:Port 格式）
        has_colon = df[col].astype(str).str.contains(":", na=False).sum()
        total_valid = df[col].notna().sum()
        
        print(f"{col}:")
        print(f"  有效值: {total_valid:,} 筆")
        print(f"  包含 ':' 的筆數: {has_colon:,} 筆 ({has_colon/total_valid*100:.2f}%)")
        
        # 顯示幾個範例
        sample_values = df[col].dropna().head(3).tolist()
        print(f"  範例值: {sample_values}")
        print()


Src_IP_Port:
  有效值: 8,087,512 筆
  包含 ':' 的筆數: 8,011,329 筆 (99.06%)
  範例值: ['188.75.133.98:16200', '119.252.172.92:59067', '90.183.101.168:80']

Dst_IP_Port:
  有效值: 8,087,512 筆
  包含 ':' 的筆數: 8,009,095 筆 (99.03%)
  範例值: ['147.32.86.125:35248', '147.32.84.14:80', '147.32.86.89:1176']



## 9. 資料品質總結與建議


In [25]:
print("=" * 60)
print("資料品質檢查總結")
print("=" * 60)

issues = []

# 檢查缺失值
if missing["missing_count"].sum() > 0:
    issues.append(f"⚠️ 發現 {missing['missing_count'].sum():,} 筆缺失值")

# 檢查負值
if negative_checks:
    issues.append(f"⚠️ 發現負值欄位: {', '.join(negative_checks.keys())}")

# 檢查零值
if zero_checks:
    issues.append(f"⚠️ 發現零值欄位: {', '.join(zero_checks.keys())}")

# 檢查標籤不平衡
if "Label" in df.columns and len(df["Label"].value_counts()) > 1:
    max_pct = df["Label"].value_counts().iloc[0] / len(df) * 100
    if max_pct > 90:
        issues.append(f"⚠️ 標籤極度不平衡（最大標籤佔 {max_pct:.1f}%）")

if not issues:
    print("✅ 資料品質良好，未發現明顯問題")
else:
    print("\n".join(issues))

print("\n建議後續處理：")
print("1. 根據缺失值分析決定填補策略")
print("2. 處理負值和零值（視業務邏輯決定是否保留）")
print("3. 針對標籤不平衡問題，考慮使用適當的評估指標（Precision, Recall, F1）")
print("4. 進行特徵工程前，先處理 IP:Port 欄位（拆分為 IP 和 Port）")
print("=" * 60)


資料品質檢查總結
⚠️ 標籤極度不平衡（最大標籤佔 90.5%）

建議後續處理：
1. 根據缺失值分析決定填補策略
2. 處理負值和零值（視業務邏輯決定是否保留）
3. 針對標籤不平衡問題，考慮使用適當的評估指標（Precision, Recall, F1）
4. 進行特徵工程前，先處理 IP:Port 欄位（拆分為 IP 和 Port）
